<a href="https://colab.research.google.com/github/jqllxew/stable-diffusion-webui/blob/master/stable_diifusion_webui_AUTOMATIC1111_jqllxew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 初始化/init

In [ ]:
from google.colab import drive
import time
import os

drive.mount('/content/gdrive')
if not os.path.exists("/content/gdrive/MyDrive/"):
  os.mkdir("/content/gdrive/MyDrive/",mode=0o755)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
repo_base_path = "/content/gdrive/MyDrive/sd"
repo_git_url = "https://github.com/jqllxew/stable-diffusion-webui.git"
project_path = repo_base_path+"/stable-diffusion-webui"

def inf(msg, style, wdth): 
  inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
  display(inf)
with capture.capture_output() as cap:
  if not os.path.exists(repo_base_path):
    os.mkdir(repo_base_path,mode=0o755)
  if not os.path.exists(project_path):
    os.chdir(repo_base_path)
    os.system(f"git clone {repo_git_url}")
  os.chdir(project_path)
  os.system(f"git pull {repo_git_url}")
  !mkdir -p {project_path}/cache/huggingface
  !ln -s {project_path}/cache/huggingface /root/.cache/
print('[1;32m')
clear_output()
inf('\u2714 Done','success', '50px')


Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

### 引用库/repositories

In [ ]:
print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists(f'{repo_base_path}/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst
  !tar -C / --zstd -xf A1111_dep.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
clear_output()
inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

### 扩展/extensions

In [ ]:
extensions_path = project_path+"/extensions"
print('[1;32mInstalling extensions...')
with capture.capture_output() as cap:
  if not os.path.exists(extensions_path):
    os.mkdir(extensions_path,mode=0o755)
  %cd {extensions_path}
  if not os.path.exists(extensions_path+"/stable-diffusion-webui-aesthetic-gradients"): # 美学优化
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients.git
  if not os.path.exists(extensions_path+"/stable-diffusion-webui-images-browser"): # 图像浏览
    !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser.git
  # if not os.path.exists(extensions_path+"/stable-diffusion-webui-nsfw-censor"): # nsfw
  #   !git clone https://github.com/jqllxew/stable-diffusion-webui-nsfw-censor.git
  if not os.path.exists(extensions_path+"/a1111-sd-webui-tagcomplete"):
    !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git
  for sub_dir in os.listdir(extensions_path):
    sub_dir_path = os.path.join(extensions_path, sub_dir)
    if os.path.isdir(sub_dir_path):
      os.chdir(sub_dir_path)
      print(f"{sub_dir_path} git pull")
      os.system("git pull")
clear_output()
inf('\u2714 Done','success', '50px')

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

### 模型下载/model download

In [ ]:
# Stable-diffusion
model_download_url = "https://huggingface.co/advokat/2dn/resolve/main/2DN-pruned-vae.ckpt" #@param {type:"string"}
%cd {project_path}/models/Stable-diffusion
!wget {model_download_url}

Button(button_style='success', description='✔ Model downloaded, using the trained model.', disabled=True, layo…

In [ ]:
!pip list|grep transformers

transformers                  4.25.1


In [ ]:
from transformers import CLIPVisionModelWithProjection

ImportError: ignored

### 运行/run

In [ ]:
project_path = "/content/gdrive/MyDrive/sd/stable-diffusion-webui"
with capture.capture_output() as cap:
  %cd {project_path}/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.8/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd {project_path}

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {project_path}/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {project_path}/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" {project_path}/modules/extras.py

%cd {project_path}
!python webui.py --share --api --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae --xformers --disable-console-progressbars

/content/gdrive/MyDrive/sd/stable-diffusion-webui
Loading weights [19b509d939] from /content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/model.ckpt
Creating model from config: /content/gdrive/MyDrive/sd/stable-diffusion-webui/configs/v1-inference.yaml
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Failed to create model quickly; will retry using slow method.
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
loading stable diffusion model: FileNotFoundError
Traceback (most recent call last):
  File "webui.py", line 111, in initialize
    modules.sd_models.load_model()
  File "/content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py", line 400, in load_model
    sd_model = instantiate_from_config(sd_config.model)
  File "/content/gdrive/MyDrive/sd/stablediffusion/ldm/util.py", line 79, in instantiate_from_config
    return get_obj_from_str(config["target"])(**config.get("params",